# Doorrekenen van een neuraal netwerk

### Inleiding
Kunstmatige neurale netwerken worden vaak beschreven door middel van matrices.
Dit maakt het makkelijk om met een neuraal netwerk te rekenen.  
Voor deze opdracht is het de bedoeling om een netwerk te kunnen inlezen vanuit een json-bestand.
Dit netwerk moet dan gebruikt worden om matrices per overgang te construeren.  
Uiteindelijk moet het de bedoeling zijn dat voor een gegeven input de output berekend wordt.

### Aanpak
Elke overgang van de ene laag naar de andere laag van een neuraal netwerk (transities)
kan worden weergegeven met een matrix $\mathbf{L}_i$.
De matrix beschrijft dan de transitie van laag $i$ naar laag $i + 1$ in het netwerk.  
Wanneer de matrices van alle transitielagen keer elkaar worden berekend kan de output van het neurale netwerk
uiteindelijk zelfs in één keer worden berekend.  
Hierbij wordt dan de volgende berekening gedaan:  
$\vec{y} = \mathbf{L}_x \cdot \vec{x}$  
Hierbij is $\vec{y}$ de uiteindelijke output vector,
$\mathbf{L}_x$ de dot product van alle transitie matrices en
$\vec{x}$ de input vector.
### Uitwerking
Allereerst wordt de json data uitgelezen uit het bestand dat is doorgegeven
in de variabele 'file_name'.  
Hierna wordt op basis van de informatie uit het bestand de input layer aangemaakt
en worden van alle lagen de namen (keys) opgeslagen.

In [5]:
import json
import numpy as np

file_name = "example-2layer.json"
with open(file_name, "r") as json_file:
    data = json.load(json_file)

# inserting the input layer
first_key = list(data.keys())[0]
input_layer = [[1.0] for x in range(0, int(data[first_key]['size_in']))]

# printing the input layer
print(np.array(input_layer))

# saving all the layer names of the json
layer_names = data.keys()


[[1.]
 [1.]
 [1.]
 [1.]
 [1.]]


Om de transities tussen de layers om te zetten naar matrices zal er
over allereerst per laag over elke node heen worden gegaan.
Per node zullen de weights van bestaande en niet bestaande (weight van 0) opgeslagen worden in een matrix.
Uiteindelijk zal er dan per transitielaag een matrix representatie worden overgehouden.


In [6]:
# list to store all the transition matrices
transition_list = []

# build up the layer_matrix
for layer in layer_names:

    layer_info = data[layer]
    layer_in = int(layer_info['size_in'])
    layer_out = int(layer_info['size_out'])
    
    # initialize a new 2d array
    some_layer = [[0 for i in range(layer_in)] for j in range(layer_out)]
    
    # select the info about the weigths
    layer_info = layer_info['weights']

    # loop over all the nodes
    for node in range(0, layer_in):
        
        # save the whole synaps dict from a node
        node_synaps_dict = layer_info[str(node + 1)]
        
        # check for every transition the weights
        for synaps in range(0, layer_out):
            
            # check if a certain synaps exists between the layers
            if str(synaps + 1) in node_synaps_dict:
                some_layer[synaps][node] = float(node_synaps_dict[str(synaps + 1)])
            else:
                continue

    # append the matrix into a list
    transition_list.append(some_layer)
    print("{0}\n".format(np.array(some_layer)))


[[ 0.5  0.2  0.   0.  -0.2]
 [ 0.2 -0.5 -0.1  0.9 -0.8]
 [ 0.   0.2  0.   0.1 -0.1]
 [ 0.1  0.8  0.3  0.  -0.7]]

[[ 0.5  0.2 -0.1  0.9]
 [ 0.2 -0.5  0.3  0.1]]



Om van alle transitiematrices één matrix te maken moeten alle matrices met elkaar vermenigvuldigt worden.  
Dit wordt gedaan door het dot product van twee matrices te nemen.
hierbij is het wel van belang om constant de volgende matrix keer de vorige matrix te nemen.
Hierdoor zullen de dimensies zich namelijk goed aanpassen.  
Wanneer bijvoorbeeld de twee matrices $\mathbf{L}_1$ en $\mathbf{L}_2$ worden vermenigvuldigt
zal de nieuwe matrix als volgt worden berekend:  
$\mathbf{L}_{1\cdot2} = \mathbf{L}_2 \cdot \mathbf{L}_1$  
Aangezien deze berekening vaker voorkomt is hiervoor een functie gemaakt:


In [3]:
def multiply_matrix(matrix_A, matrix_B):
    """Takes two matrices and returns the result of matrix_A times matrix_B"""
    
    multi_matrix = [[0 for i in range(len(matrix_B[0]))] for j in range(len(matrix_A))]
    
    # loop over every row in matrix A
    for i in range(0, len(matrix_A)):
        
        # loop over every column in matrix B
        for j in range(0, len(matrix_B[0])):
            
            # loop over every element in the row of A and column of B
            for k in range(0, len(matrix_A[0])):
                multi_matrix[i][j] += matrix_A[i][k] * matrix_B[k][j]
                # print("{0} * {1}".format(matrix_A[i][k], matrix_B[k][j]))
    return multi_matrix


Nadat alle transitiematrices aangemaakt zijn kunnen ze met elkaar vermenigvuldigt worden waar nodig.  
Allereerst moet gecheckt worden hoeveel transitiematrices er zijn gemaakt.  
Dit aangezien wanneer er maar één transitie plaatsvind deze matrix niet met een 
andere matrix vermenigvuldigt hoeft te worden om tot DE transitiematrix te komen voor het gehele netwerk.
Wanneer er wel meerdere transitiematrices aanwezig zijn zullen ze allemaal
vermenigvuldigt worden met elkaar volgens het eerder uitgelegde principe van
$\mathbf{L}_{1\cdot2} = \mathbf{L}_2 \cdot \mathbf{L}_1$.  
Uiteindelijk zal de complete transitiematrix vermenigvuldigt worden met de input layer
waarnaar de output layer als resultaat eruit zal komen:   
$\vec{o} = \mathbf{L}_x \cdot \vec{i}$  
Waarbij $\vec{0}$ de output vector is, $\vec{i}$ de input vector en
$\mathbf{L}_x$ de uiteindelijke transitie vector.


In [4]:
# check how many transition layers there are
if len(transition_list) == 1:
    
    # there exists only 1 transition matrix
    output_layer = multiply_matrix(transition_list[0], input_layer)
else:

    # multiple transition layers
    output_layer = transition_list[0]
    for transition in range(1, len(transition_list)):

        # calculate the cross product
        output_layer = multiply_matrix(transition_list[transition], output_layer)
    
    # multiply the calculated transition matrix with the input layer
    output_layer = multiply_matrix(output_layer, input_layer)

# printing the output
print("The output layer is:\n{0}".format(np.array(output_layer)))



The output layer is:
[[0.62]
 [0.36]]
